In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tqdm import tqdm_notebook

In [3]:
df = pd.read_csv('data/train.csv')
df = df.rename({'Company ': 'Company'}, axis=1)
df = df.sort_values(['Company', 'ID'])
df = df.reset_index()

features = df.columns[4:-1].tolist()
previous_records_count = 10
next_records_count = 10

for feature in features:
    for i in range(-previous_records_count, next_records_count + 1):
        if i != 0:
            df.insert(df.shape[1], '{0}_{1}'.format(feature, i), np.nan)

for index, row in tqdm_notebook(df.iterrows(), total=df.shape[0], desc='Filling additional features'):
    for feature in features:
        for i in range(-previous_records_count, next_records_count + 1):
            if i != 0:
                if index + i > 0 and index + i < df.shape[0] and df.at[index + i, 'Company'] == row['Company']:
                    df.at[index, '{0}_{1}'.format(feature, i)] = df.loc[index + i, feature]

df = df.dropna()
df[:5]

,index,ID,Date,Company,SMA,EMA,WMA,DEMA,TEMA,TRIMA,...,QUADRATURE_1,QUADRATURE_2,QUADRATURE_3,QUADRATURE_4,QUADRATURE_5,QUADRATURE_6,QUADRATURE_7,QUADRATURE_8,QUADRATURE_9,QUADRATURE_10
64,192,193,20-04-2148,ABC,26.773,27.2556,27.2455,27.8216,28.0471,26.5270,...,5.0483,4.1573,2.0008,-2.6105,-3.6381,-1.6362,-2.3343,-1.9080,-0.6846,-0.0651
65,195,196,21-04-2148,ABC,26.979,27.5837,27.6613,28.3152,28.6351,26.9583,...,4.1573,2.0008,-2.6105,-3.6381,-1.6362,-2.3343,-1.9080,-0.6846,-0.0651,1.0375
66,198,199,22-04-2148,ABC,27.273,27.7957,27.9833,28.5677,28.8627,27.4627,...,2.0008,-2.6105,-3.6381,-1.6362,-2.3343,-1.9080,-0.6846,-0.0651,1.0375,0.9174
67,201,202,23-04-2148,ABC,27.492,27.7311,28.0136,28.3098,28.3929,27.8607,...,-2.6105,-3.6381,-1.6362,-2.3343,-1.9080,-0.6846,-0.0651,1.0375,0.9174,1.0634
68,204,205,24-04-2148,ABC,27.761,27.7345,28.0605,28.2108,28.1951,28.0983,...,-3.6381,-1.6362,-2.3343,-1.9080,-0.6846,-0.0651,1.0375,0.9174,1.0634,1.5920


In [4]:
corr = df[df.columns[3:]].corr().sort_values('Price', ascending=False)['Price']
corr

Price                 1.000000
TEMA_2                0.999731
DEMA_3                0.999658
TRIMA_6               0.999640
DEMA_2                0.999600
TEMA_3                0.999586
WMA_4                 0.999575
TRIMA_5               0.999555
TRIMA_7               0.999537
WMA_5                 0.999510
WMA_3                 0.999495
Real Middle Band_6    0.999482
SMA_6                 0.999482
DEMA_4                0.999477
TEMA_1                0.999443
SMA_7                 0.999428
Real Middle Band_7    0.999428
Real Middle Band_5    0.999426
SMA_5                 0.999426
T3_7                  0.999417
T3_8                  0.999395
EMA_4                 0.999380
EMA_5                 0.999371
T3_6                  0.999315
WMA_6                 0.999315
TRIMA_4               0.999295
EMA_3                 0.999276
TRIMA_8               0.999265
EMA_6                 0.999264
Real Middle Band_4    0.999256
                        ...   
LEAD SINE            -0.076257
LEAD SIN

In [7]:
model_vars = [name for name in corr.index if corr[name] < 1.0 and corr[name] > 0.1]
model = sm.OLS(df['Price'], df[model_vars]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.866e+05
Date:                Thu, 04 Jul 2019   Prob (F-statistic):               0.00
Time:                        16:08:52   Log-Likelihood:                -17076.
No. Observations:               11775   AIC:                         3.493e+04
Df Residuals:                   11388   BIC:                         3.778e+04
Df Model:                         386                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
TEMA_2                 164.2229    267.027      0.615      0.539    -359.196     687.642
DEMA_3                  67.2206    100.402      0.670      0.503    -129.585     264.026
TRIMA_6               -284.3245    357.445     -0.795      0.426    -984.979     416.330
DEMA_2                 -77.8242     98.139     -0.793      0.428    -270.194     114.545
TEMA_3                -187.3004    272.016     -0.689      0.491    -720.498     345.898
WMA_4                 -357.8356    333.658     -1.072      0.284   -1011.863     296.192
TRIMA_5                 62.9710    356.511      0.177      0.860    -635.853     761.795
TRIMA_7               -662.1212    358.234     -1.848      0.065   -1364.321      40.079
WMA_5                  742.0140    333.086      2.228      0.026      89.108    1394.920
WMA_3                 -481.3256    334.461     -1.439      0.150   -1136.928     174.276
Real Middle Band_6    -105.5264    276.157     -0.382      0.702    -646.842     435.789
SMA_6                 -103.7500    276.245     -0.376      0.707    -645.237     437.737
DEMA_4                  55.1529    102.314      0.539      0.590    -145.401     255.707
TEMA_1                 -64.5605    261.294     -0.247      0.805    -576.743     447.622
SMA_7                  558.5262    283.282      1.972      0.049       3.244    1113.808
Real Middle Band_7     557.5448    283.232      1.969      0.049       2.362    1112.728
Real Middle Band_5    -432.3379    293.672     -1.472      0.141   -1007.985     143.309
SMA_5                 -431.7837    293.554     -1.471      0.141   -1007.200     143.633
T3_7                  -162.8507    327.452     -0.497      0.619    -804.713     479.012
T3_8                   186.7236    327.412      0.570      0.568    -455.060     828.507
EMA_4                  -57.6951    331.157     -0.174      0.862    -706.820     591.429
EMA_5                   35.1611    331.841      0.106      0.916    -615.304     685.626
T3_6                  -227.6093    327.505     -0.695      0.487    -869.575     414.356
WMA_6                  144.3924    333.069      0.434      0.665    -508.479     797.264
TRIMA_4               -602.6947    357.704     -1.685      0.092   -1303.856      98.467
EMA_3                  417.9934    331.266      1.262      0.207    -231.345    1067.332
TRIMA_8                414.8263    357.760      1.160      0.246    -286.444    1116.097
EMA_6                 -464.3387    331.023     -1.403      0.161   -1113.200     184.523
Real Middle Band_4     498.2360    279.013      1.786      0.074     -48.678    1045.150
SMA_4                  498.3831    279.020      1.786      0.074     -48.544    1045.310
SMA_8                    0.2425    294.591      0.001      0.999    -577.206     577.691
Real Middle Band_8       0.1169    294.588      0.000    

In [8]:
full_names = ['simpleMovingAverage', 'exponentialMovingAverage', 'waysAndMeansAdvances', 'doubleExponentialMovingAverage', 'tripleExponentialMovingAverage', 'triangularMovingAverage', 'kaufmansAdaptiveMovingAverage', 'fAdaptiveMovingAverage', 'mesaAdaptiveMovingAverage', 't3MovingAverage', 'movingAverageConvergenceDivergence', 'movingAverageConvergenceDivergenceHistogram', 'movingAverageConvergenceDivergenceSignal', 'movingAverageConvergence', 'movingAverageConvergenceHistogram', 'movingAverageConvergenceSignal', 'slowStochasticOscillatorSecondLine', 'slowStochasticOscillatorMainLine', 'fastStochasticOscillatorSecondLine', 'fastStochasticOscillatorMainLine', 'relativeStrengthIndex', 'fastStochasticSecondLine', 'fastStochasticMainLine', 'willr', 'averageDirectionalIndex', 'averageDirectionalIndexRating', 'absolutePriceOscillator', 'percentagePriceOscillator', 'momentumIndicator', 'balanceOfPower', 'commodityChannelIndex', 'chandeMomentumOscillator', 'rateOfChange', 'rateOfChangeRating', 'aroonOscillatorDown', 'aroonOscillatorUp', 'aroonOscillator', 'moneyFlowIndex', 'tripleSmoothedExponentialMovingAverage', 'ultimateOscillatorDefinitionAndStrategies', 'directionalMovementIndex', 'minusDirectionalIndex', 'plusDirectionalIndex', 'minusDirectionalMovementIndex', 'plusDirectionalMovementIndex', 'relativeStrengthIndexLowerBand', 'relativeStrengthIndexMiddleBand', 'relativeStrengthIndexHighBand', 'midPoint', 'midPrice', 'parabolicStopAndReverse', 'trange', 'averageTrueRange', 'normalizedAverageTrueRange', 'chaikinAccumulationDistributionLine', 'accumulationDistributionOscillatorSC', 'onBalanceVolume', 'htTrendline', 'leadSine', 'sine', 'trendMode', 'dcPeriod', 'htDcPhase', 'phase', 'quadrature']

indexes = [int(name.rsplit('_', 1)[1]) for name in model.params.index if '_' in name and name not in features]

if len(indexes) > 0:
    if min(indexes) < 0:
        print('if (previousRecords.size < {0}) {{'.format(max(indexes)))
        print('    return record.tripleExponentialMovingAverage')
        print('}')
        print()
    if max(indexes) > 0:
        print('if (nextRecords.size < {0}) {{'.format(max(indexes)))
        print('    return record.tripleExponentialMovingAverage')
        print('}')
        print()

print('return listOf(')
    
for name in model.params.index:
    modifier = model.params[name]
    comma = ',' if name != model.params.index[-1] else ''
    
    if '_' in name and name not in features:
        parts = name.rsplit('_', 1)
        recordsList = 'previousRecords' if int(parts[1]) < 0 else 'nextRecords'
        index = abs(int(parts[1])) - 1
        full_name = full_names[features.index(parts[0])]
        
        print('    {0}[{1}].{2} * {3}{4}'.format(recordsList, index, full_name, modifier, comma))
    else:
        full_name = full_names[features.index(name)]
        
        print('    record.{0} * {1}{2}'.format(full_name, modifier, comma))

print(').sum()')

if (previousRecords.size < 10) {
    return record.tripleExponentialMovingAverage
}

if (nextRecords.size < 10) {
    return record.tripleExponentialMovingAverage
}

return listOf(
    nextRecords[1].tripleExponentialMovingAverage * 164.22290462325327,
    nextRecords[2].doubleExponentialMovingAverage * 67.22058958073285,
    nextRecords[5].triangularMovingAverage * -284.32446424924274,
    nextRecords[1].doubleExponentialMovingAverage * -77.82423750441103,
    nextRecords[2].tripleExponentialMovingAverage * -187.3004331245229,
    nextRecords[3].waysAndMeansAdvances * -357.83558501654625,
    nextRecords[4].triangularMovingAverage * 62.97104934809249,
    nextRecords[6].triangularMovingAverage * -662.1211581128373,
    nextRecords[4].waysAndMeansAdvances * 742.0139793790877,
    nextRecords[2].waysAndMeansAdvances * -481.32562016197335,
    nextRecords[5].relativeStrengthIndexMiddleBand * -105.52639376140723,
    nextRecords[5].simpleMovingAverage * -103.74997403107045,
    nextRecord

    previousRecords[8].tripleExponentialMovingAverage * 315.3324232683899,
    previousRecords[6].simpleMovingAverage * 219.50817539831405,
    previousRecords[6].relativeStrengthIndexMiddleBand * 219.82392974990216,
    previousRecords[5].kaufmansAdaptiveMovingAverage * -0.01211880033116186,
    previousRecords[7].exponentialMovingAverage * -519.0814830373856,
    previousRecords[8].doubleExponentialMovingAverage * -51.485104467967176,
    previousRecords[7].midPrice * -0.0034476540260870436,
    previousRecords[5].t3MovingAverage * 152.39404243436002,
    previousRecords[6].triangularMovingAverage * -187.46828478597126,
    previousRecords[7].midPoint * 0.03186932863291836,
    previousRecords[6].relativeStrengthIndexLowerBand * -24.51310556596036,
    previousRecords[1].htTrendline * -0.8940929601450343,
    previousRecords[8].waysAndMeansAdvances * 68.49749275091744,
    nextRecords[7].parabolicStopAndReverse * 0.016053917361893433,
    previousRecords[9].tripleExponentialMovingAve

    nextRecords[2].movingAverageConvergenceDivergenceSignal * -500.1494128523864,
    nextRecords[1].movingAverageConvergenceDivergenceSignal * 340.9658974712802,
    nextRecords[3].movingAverageConvergenceDivergenceSignal * 82.60336567433842,
    nextRecords[0].movingAverageConvergenceDivergenceSignal * -58.99789305851482,
    nextRecords[4].movingAverageConvergenceDivergenceSignal * 50.356432461805525,
    record.movingAverageConvergenceDivergenceSignal * 259.2400359595813,
    previousRecords[0].movingAverageConvergenceDivergenceSignal * 85.52549283747612,
    previousRecords[1].movingAverageConvergenceDivergenceSignal * -69.66559601827794,
    nextRecords[5].movingAverageConvergenceDivergenceSignal * -351.9993704506378,
    previousRecords[2].movingAverageConvergenceDivergenceSignal * -86.52861349217346,
    previousRecords[3].movingAverageConvergenceDivergenceSignal * 741.4757704976992,
    previousRecords[4].movingAverageConvergenceDivergenceSignal * 495.58800556231654,
    nextR